In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as T
from torchvision.transforms import functional as F
import random
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import DataLoader, random_split, Dataset
from torchmetrics.image import PeakSignalNoiseRatio, StructuralSimilarityIndexMeasure
from tqdm import tqdm
import warnings
import wandb
import torch.optim.lr_scheduler as lr_scheduler
import os
import torchvision.transforms.v2 as transforms
from PIL import Image

warnings.filterwarnings("ignore")
torch.manual_seed(42)
np.random.seed(42)

In [3]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("WANDB_API_KEY")

wandb.login(key = secret_value_0)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: shobhitshukla6535 (shobhitshukla6535-iit-kharagpur). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
class ImageDenoisingDataset(Dataset):
    def __init__(self, noisy_dir, original_dir, patch_size=50, stride=25):
        self.noisy_dir = noisy_dir
        self.original_dir = original_dir
        self.patch_size = patch_size
        self.stride = stride
        self.transform = transforms.Compose(
            [transforms.ToImage(), transforms.ToDtype(torch.float32, scale=True)]
        )

        self.noisy_images = sorted(os.listdir(noisy_dir))
        self.original_images = sorted(os.listdir(original_dir))

        assert len(self.noisy_images) == len(self.original_images), (
            "Mismatch in dataset sizes"
        )

        self.patches = self._generate_patches()

    def _generate_patches(self):
        """Generate all patches using a sliding window approach."""
        patches = []

        for img_idx in range(len(self.noisy_images)):
            noisy_path = os.path.join(self.noisy_dir, self.noisy_images[img_idx])
            original_path = os.path.join(
                self.original_dir, self.original_images[img_idx]
            )

            noisy_img = Image.open(noisy_path).convert("RGB")
            original_img = Image.open(original_path).convert("RGB")

            assert noisy_img.size == original_img.size, (
                f"Image size mismatch for {img_idx}"
            )

            w, h = noisy_img.size
            ps = self.patch_size
            stride = self.stride

            # Extract patches using a sliding window approach
            for y in range(0, h - ps + 1, stride):
                for x in range(0, w - ps + 1, stride):
                    patches.append((img_idx, x, y))

        return patches

    @staticmethod
    def apply_same_transforms(noisy_img, original_img):
        seed = random.randint(0, 2**32)

        def get_transforms(seed):
            random.seed(seed)
            torch.manual_seed(seed)

            angle = random.uniform(-180, 180)
            flip = random.random() > 0.5

            def transform(img):
                img = F.rotate(img, angle)

                if flip:
                    img = F.hflip(img)

                return img

            return transform

        transform = get_transforms(seed)

        transformed_noisy = transform(noisy_img)
        transformed_original = transform(original_img)

        random.seed(42)
        torch.manual_seed(42)
        
        return transformed_noisy, transformed_original

    
    def __len__(self):
        return len(self.patches)


    def __getitem__(self, idx):
        """Return a specific patch from the dataset."""
        img_idx, x, y = self.patches[idx]

        noisy_path = os.path.join(self.noisy_dir, self.noisy_images[img_idx])
        original_path = os.path.join(self.original_dir, self.original_images[img_idx])

        noisy_img = Image.open(noisy_path).convert("RGB")
        original_img = Image.open(original_path).convert("RGB")

        noisy_patch = noisy_img.crop((x, y, x + self.patch_size, y + self.patch_size))
        original_patch = original_img.crop(
            (x, y, x + self.patch_size, y + self.patch_size)
        )

        transformed_noisy, transformed_original = self.apply_same_transforms(
            noisy_patch, original_patch
        )

        transformed_noisy_tensor = self.transform(transformed_noisy)
        transformed_original_tensor = self.transform(transformed_original)

        return {
            "noisy_patch": transformed_noisy_tensor,
            "original_patch": transformed_original_tensor,
        }

In [5]:
class DnCNN(nn.Module):
    def __init__(self, depth=20, channels=128):
        super(DnCNN, self).__init__()

        layers = [
            nn.Conv2d(
                in_channels=3,
                out_channels=channels,
                kernel_size=3,
                padding=1,
                bias=False,
            ),
            nn.ReLU(inplace=True),
        ]

        for _ in range(depth - 2):
            layers.append(
                nn.Conv2d(
                    in_channels=channels,
                    out_channels=channels,
                    kernel_size=3,
                    padding=1,
                    bias=False,
                )
            )
            layers.append(nn.BatchNorm2d(channels))
            layers.append(nn.ReLU(inplace=True))

        layers.append(
            nn.Conv2d(
                in_channels=channels,
                out_channels=3,
                kernel_size=3,
                padding=1,
                bias=False,
            )
        )

        self.dncnn = nn.Sequential(*layers)

    def forward(self, x):
        noise = self.dncnn(x)
        return noise

In [ ]:
class Trainer:
    def __init__(self, config):
        self.config = config
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self._prepare_dataloader()
        self._init_model()
        os.makedirs(self.config["checkpoint_dir"], exist_ok=True)

        self.optimizer = optim.AdamW(
            self.model.parameters(), lr=1e-4
        )
        
        self.scheduler = lr_scheduler.ExponentialLR(self.optimizer, gamma=0.99)
        self.criterion = nn.MSELoss()

        self.train_psnr = PeakSignalNoiseRatio().to(self.device)
        self.val_psnr = PeakSignalNoiseRatio().to(self.device)
        self.train_ssim = StructuralSimilarityIndexMeasure().to(self.device)
        self.val_ssim = StructuralSimilarityIndexMeasure().to(self.device)

        if self.config["wandb_enabled"] and self.config["wandb_mode"] != "disabled":
            exclude_keys = [
                "project_name",
                "wandb_mode",
                "wandb_enabled",
                "noisy_dir",
                "original_dir",
                "val_size",
            ]
            
            parsed_config = wandb.helper.parse_config(
                self.config, 
                exclude=exclude_keys
            )

            self.wandb_run = wandb.init(
                project=self.config["project_name"],
                config=parsed_config,
                mode=self.config["wandb_mode"],
            )
            wandb.watch(models=self.model, criterion=self.criterion, log="gradients")
        else:
            self.wandb_run = None

    def _prepare_dataloader(self):
        print("Preparing Data...")
        dataset = ImageDenoisingDataset(
            noisy_dir=self.config["noisy_dir"],
            original_dir=self.config["original_dir"],
            patch_size=self.config["patch_size"],
            stride=self.config["stride"],
        )

        assert torch.initial_seed() == 42
        
        dataset_size = len(dataset)
        val_size = int(self.config["val_size"] * dataset_size)
        train_size = dataset_size - val_size

        train_data, val_data = random_split(dataset, [train_size, val_size])

        self.train_loader = DataLoader(
            train_data, batch_size=self.config["batch_size"], shuffle=True
        )
        self.val_loader = DataLoader(val_data, batch_size=self.config["batch_size"])
        print("Dataloaders prepared !!")

    def _init_model(self):
        self.model = DnCNN(depth=self.config["depth"], channels=self.config["channels"]).to(self.device)
        print("Model Initialized !!")

    def train(self):
        
        best_val_psnr = 0
        step = 0
        agg_loss = 0

        for epoch in range(self.config["epochs"]):
            
            self.train_psnr.reset()
            self.train_ssim.reset()

            self.model.train()
            batch_losses = []

            batch_iterator = tqdm(
                self.train_loader,
                desc=f"Epoch {epoch + 1}",
                leave=True,
                dynamic_ncols=True,
            )

            for batch in batch_iterator:
                y = batch["noisy_patch"].to(self.device)
                x = batch["original_patch"].to(self.device)
                target_residual = y - x

                self.optimizer.zero_grad()

                output_residual = self.model(y)
                
                denoised_image = y - output_residual

                loss = self.criterion(output_residual, target_residual)
                loss.backward()

                nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)

                self.optimizer.step()
                agg_loss += loss.item()

                self.train_psnr.update(denoised_image, x)
                self.train_ssim.update(denoised_image, x)

                if self.wandb_run:
                    wandb.log(
                        {"avg loss": agg_loss / (step + 1), "training_step": step},
                        step=step,
                    )
                step += 1

                batch_losses.append(loss.item())

                batch_iterator.set_postfix({"Avg Loss": agg_loss / (step + 1)})
                
            train_loss = sum(batch_losses)/len(self.train_loader)
            self.scheduler.step()
            print(f"Latest Learning rate: {self.scheduler.get_last_lr()}")


            if (epoch+1) % self.config["log_interval"] == 0:
                # Get metric values
                train_psnr = self.train_psnr.compute()
                train_ssim = self.train_ssim.compute()

                val_loss, val_psnr, val_ssim = self.validate()

                if self.wandb_run:
                    wandb.log(
                        {
                            "epoch": epoch + 1,
                            # "training_step": step,
                            "train_loss": train_loss,
                            "train_psnr": train_psnr.item(),
                            "train_ssim": train_ssim.item(),
                            "val_loss": val_loss,
                            "val_psnr": val_psnr,
                            "val_ssim": val_ssim
                        }
                    )
                print(
                    f"Epoch {epoch + 1}/{self.config['epochs']}, Step: {step}, "
                    f"Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}, "
                    f"Train PSNR: {train_psnr.item():.6f}, Train SSIM: {train_ssim.item():.6f}, "
                    f"Val PSNR: {val_psnr:.6f}, Val SSIM: {val_ssim:.6f}"
                )

                if val_psnr > best_val_psnr:
                    best_val_psnr = val_psnr
                    print(
                        f"New best model found at epoch {epoch + 1} with val PSNR: {best_val_psnr:.6f}"
                    )
                    model_path = f"{self.config['checkpoint_dir']}/best_model.pth"
                    torch.save(self.model.state_dict(), model_path)
                    if self.wandb_run:
                        wandb.log({"best_val_psnr": best_val_psnr}, step=step)
                        wandb.save(model_path)
                        self._log_model_as_artifact(model_path)

        if self.wandb_run:
            wandb.finish()
        print("Training completed.")

    def validate(self):
        self.model.eval()
        self.val_psnr.reset()
        self.val_ssim.reset()
        total_loss = 0

        with torch.no_grad():
            for batch in self.val_loader:
                y = batch["noisy_patch"].to(self.device)
                x = batch["original_patch"].to(self.device)
                target_residual = y - x

                outputs_residual = self.model(y)
                # Reconstruct the denoised image
                denoised_image = y - outputs_residual

                loss = self.criterion(outputs_residual, target_residual)
                total_loss += loss.item()

                self.val_psnr.update(denoised_image, x)
                self.val_ssim.update(denoised_image, x)
                
        val_loss = total_loss / len(self.val_loader)
        return (
            val_loss,
            self.val_psnr.compute().item(),
            self.val_ssim.compute().item(),
        )

    def _log_model_as_artifact(self, model_path):
        artifact = wandb.Artifact(f"best_model", type="model")
        artifact.add_file(model_path)
        wandb.log_artifact(artifact)

In [11]:
config = {
    "learning_rate": 1e-4,  # Learning rate for the AdamW optimizer
    "batch_size": 128,  # Batch size for training and validation
    "epochs": 50,  # Number of training epochs
    "depth": 20,   # Depth of the DnCNN model
    "channels": 128 ,
    "patch_size": 50,  # Size of the patches extracted from images
    "stride": 25,  # Stride for the sliding window patch extraction
    "val_size": 0.2,  # Fraction of data used for validation
    "checkpoint_dir": "./checkpoints",  # Directory to save model checkpoints
    "log_interval": 1,  # Log results every 1 epochs
    "wandb_enabled": True,  # Enable Weights & Biases logging
    "wandb_mode": "online",  # "online", "offline", or "disabled"
    "project_name": "ImageDenoising_1",  # Project name for Weights & Biases
    "noisy_dir": "/kaggle/input/cbsd68/noisy35",  # Directory containing noisy images
    "original_dir": "/kaggle/input/cbsd68/original_png",  # Directory containing original images
}

In [12]:
trainer = Trainer(config=config)

Preparing Data...
Dataloaders prepared !!
Model Initialized !!


In [13]:
len(trainer.train_loader), len(trainer.val_loader)

(85, 22)

In [14]:
trainer.train()

Epoch 1: 100%|██████████| 85/85 [02:41<00:00,  1.90s/it, Avg Loss=0.0227]


Latest Learning rate: [9.900000000000001e-05]
Epoch 1/50, Step: 85, Train Loss: 0.023007, Val Loss: 0.014008, Train PSNR: 16.365007, Train SSIM: 0.471272, Val PSNR: 18.532881, Val SSIM: 0.462939
New best model found at epoch 1 with val PSNR: 18.532881


Epoch 2: 100%|██████████| 85/85 [02:39<00:00,  1.88s/it, Avg Loss=0.0177]


Latest Learning rate: [9.801e-05]
Epoch 2/50, Step: 170, Train Loss: 0.012631, Val Loss: 0.010696, Train PSNR: 18.978928, Train SSIM: 0.463005, Val PSNR: 19.697124, Val SSIM: 0.485997
New best model found at epoch 2 with val PSNR: 19.697124


Epoch 3: 100%|██████████| 85/85 [02:32<00:00,  1.79s/it, Avg Loss=0.0145]


...

Latest Learning rate: [6.111172395328652e-05]
Epoch 49/50, Step: 4165, Train Loss: 0.001630, Val Loss: 0.001641, Train PSNR: 27.873386, Train SSIM: 0.839805, Val PSNR: 27.787205, Val SSIM: 0.839647
New best model found at epoch 49 with val PSNR: 27.787205


Epoch 50: 100%|██████████| 85/85 [02:30<00:00,  1.77s/it, Avg Loss=0.00293]


Latest Learning rate: [6.050060671375366e-05]
Epoch 50/50, Step: 4250, Train Loss: 0.001625, Val Loss: 0.001639, Train PSNR: 27.886539, Train SSIM: 0.840293, Val PSNR: 27.794825, Val SSIM: 0.840311
New best model found at epoch 50 with val PSNR: 27.794825


avg loss,█▇▅▄▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
best_val_psnr,▁▂▃▄▅▆▆▆▆▆▇▇▇▇▇▇▇███████████████████████
epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train_loss,█▅▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_psnr,▁▃▄▅▅▆▆▆▇▇▇▇▇▇▇▇▇███████████████████████
train_ssim,▁▁▂▄▄▆▆▆▆▆▇▇▇▇▇▇▇▇██████████████████████
training_step,▁▁▁▁▂▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇███
val_loss,█▆▄▃▄▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_psnr,▁▂▃▄▃▆▆▆▆▆▆▇▇▇▇▇▇▇██████████████████████
val_ssim,▁▁▃▄▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇█████████████████████
avg loss,0.00293


Training completed.


In [15]:
config = {
    "learning_rate": 1e-4,  # Learning rate for the AdamW optimizer
    "batch_size": 64,  # Batch size for training and validation
    "epochs": 50,  # Number of training epochs
    "depth": 20,   # Depth of the DnCNN model
    "channels": 128 ,
    "patch_size": 50,  # Size of the patches extracted from images
    "stride": 25,  # Stride for the sliding window patch extraction
    "val_size": 0.2,  # Fraction of data used for validation
    "checkpoint_dir": "./checkpoints",  # Directory to save model checkpoints
    "log_interval": 1,  # Log results every 1 epochs
    "wandb_enabled": True,  # Enable Weights & Biases logging
    "wandb_mode": "online",  # "online", "offline", or "disabled"
    "project_name": "ImageDenoising_1",  # Project name for Weights & Biases
    "noisy_dir": "/kaggle/input/cbsd68/noisy35",  # Directory containing noisy images
    "original_dir": "/kaggle/input/cbsd68/original_png",  # Directory containing original images
}

In [16]:
trainer = Trainer(config=config)

Preparing Data...
Dataloaders prepared !!
Model Initialized !!


In [17]:
len(trainer.train_loader), len(trainer.val_loader)

(169, 43)

In [18]:
trainer.train()

Epoch 1: 100%|██████████| 169/169 [02:31<00:00,  1.12it/s, Avg Loss=0.0185]


Latest Learning rate: [9.900000000000001e-05]
Epoch 1/50, Step: 169, Train Loss: 0.018602, Val Loss: 0.012042, Train PSNR: 17.298246, Train SSIM: 0.465859, Val PSNR: 19.190409, Val SSIM: 0.466052
New best model found at epoch 1 with val PSNR: 19.190409


Epoch 2: 100%|██████████| 169/169 [02:31<00:00,  1.12it/s, Avg Loss=0.0132]


Latest Learning rate: [9.801e-05]
Epoch 2/50, Step: 338, Train Loss: 0.007961, Val Loss: 0.005318, Train PSNR: 20.984205, Train SSIM: 0.539370, Val PSNR: 22.736805, Val SSIM: 0.603852
New best model found at epoch 2 with val PSNR: 22.736805


Epoch 3: 100%|██████████| 169/169 [02:33<00:00,  1.10it/s, Avg Loss=0.0103]


...

Latest Learning rate: [6.111172395328652e-05]
Epoch 49/50, Step: 8281, Train Loss: 0.001525, Val Loss: 0.001556, Train PSNR: 28.165258, Train SSIM: 0.847636, Val PSNR: 28.047297, Val SSIM: 0.852337
New best model found at epoch 49 with val PSNR: 28.047297


Epoch 50: 100%|██████████| 169/169 [02:34<00:00,  1.09it/s, Avg Loss=0.00231]


Latest Learning rate: [6.050060671375366e-05]
Epoch 50/50, Step: 8450, Train Loss: 0.001521, Val Loss: 0.001554, Train PSNR: 28.177935, Train SSIM: 0.848126, Val PSNR: 28.051971, Val SSIM: 0.852527
New best model found at epoch 50 with val PSNR: 28.051971


avg loss,█▄▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
best_val_psnr,▁▅▆▆▆▇▇▇▇▇▇▇████████████████████████████
epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇██
train_loss,█▄▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_psnr,▁▃▆▆▆▇▇▇▇▇▇▇████████████████████████████
train_ssim,▁▂▄▅▆▇▇▇▇▇▇▇▇▇▇█████████████████████████
training_step,▁▁▁▁▂▂▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇████
val_loss,█▄▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_psnr,▁▄▅▆▆▇▇▇▇▇▇▇▇███████████████████████████
val_ssim,▁▅▆▆▆▇▇▇▇▇▇▇████████████████████████████
avg loss,0.00231


Training completed.
